In [ ]:
!pip install pdfplumber
!pip install lightfm # 하이브리드 추천 시스템을 위한 대표 라이브러리
!pip install --upgrade gspread google-auth # 구글 시트-코랩 연동을 위한 라이브러리

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp311-cp311-linux_x86_64.whl size=831162 sha256=edecf9b8b808b4eb4bd816f2f2b07967c6ae239bb2866e85928939106c74c749
  Stored in directory: /root/.cache/pip/wheels/b9/0d/8a/0729d2e6e3ca2a898ba55201f905da7db3f838a33df5b3fcdd
Successfully built lightfm
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.38.0
    Uninstalling go

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.font_manager as fm
import warnings
from sentence_transformers import SentenceTransformer
from scipy.sparse import csr_matrix

from lightfm import LightFM
from lightfm.data import Dataset
from scipy.sparse import coo_matrix, csr_matrix

from bs4 import BeautifulSoup
import requests
import io
import pdfplumber
from urllib.parse import urljoin

from google.colab import auth
auth.authenticate_user()

import google.auth
import gspread
from oauth2client.client import GoogleCredentials
from google.oauth2.service_account import Credentials
import re

warnings.simplefilter("ignore")

import logging
logging.getLogger("pdfminer.pdfpage").setLevel(logging.ERROR)

In [ ]:
recruit_df = pd.read_csv("/content/서울시 동대문구 채용 정보.csv", encoding='euc-kr', low_memory=False)

# '제목' 컬럼에 '합격', '면접', '결과' 단어가 들어간 행을 필터링하여 제거
recruit_df = recruit_df[~recruit_df['제목'].str.contains('합격|면접|결과', na=False)]
recruit_df = recruit_df.drop("일련번호", axis=1)
recruit_df = recruit_df.reset_index(drop=True)

recruit_df[:20]

FileNotFoundError: [Errno 2] No such file or directory: '/content/서울시 동대문구 채용 정보.csv'

In [ ]:
recruit_df.info()

In [ ]:
link_list = recruit_df['링크'].tolist()
link_list

# 필요한 정보
# 1. user_text: 유저 자신의 취업 정보와 관련된 문장
# 2. item_text: 채용 공고 문장
# 3. interaction_data: 유저가 관심있는 채용 공고 인덱스


In [ ]:
# 2. item_text : '링크' 열의 채용 공고 pdf 파일의 텍스트 추출
item_texts = []
total_links = len(link_list)

for idx, i in enumerate(link_list, start=1):
    try:
        base_url = i  # 절대 url

        # headers 추가해서 html 가져오기
        headers = {"User-Agent": "Mozilla/5.0"}
        r = requests.get(base_url, headers=headers)

        # HTML 파싱
        soup = BeautifulSoup(r.text, 'html.parser')

        # downloadBbsFile.do 가 포함된 모든 <a> 태그 중, 내부 텍스트에 "공고"가 포함된 태그 찾기
        link_tag = None
        for tag in soup.find_all('a', href=lambda x: x and 'downloadBbsFile.do' in x):
            if '공고' in tag.get_text():
                link_tag = tag
                break

        # 상대 주소 및 절대 URL 구성
        relative_url = link_tag['href']
        full_url = urljoin(base_url, relative_url)

        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(full_url, headers=headers)

        # PDF 파일 다운로드 (메모리 상에서 처리)
        pdf_bytes = io.BytesIO(response.content)

        # pdfplumber로 텍스트 추출
        text = ""
        with pdfplumber.open(pdf_bytes) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
        item_texts.append(text)
    except Exception as e:
        # 예외 발생 시, recruit_df에서 제목, 내용, 본문 텍스트 추출
        base_url = i  # 절대 url
        headers = {"User-Agent": "Mozilla/5.0"}
        r = requests.get(base_url, headers=headers)
        soup = BeautifulSoup(r.text, 'html.parser')
        content_element = soup.find("td", class_="p-table__content", title="내용")
        main_text = content_element.get_text(separator="\n", strip=True) if content_element else ""
        row = recruit_df[recruit_df['링크'] == i]
        text = str(row['제목'].iloc[0]) + " " + str(row['내용'].iloc[0]) + " " + main_text
        item_texts.append(text)

    # 10개마다 진행 상황 출력
    if idx % 10 == 0:
        print(f"진행 상황: {total_links}개 중 {idx}개 처리됨.")


In [ ]:
# item_texts를 pickle 객체로 저장

import pickle

with open("item_texts.pkl", "wb") as f:
    pickle.dump(item_texts, f)

In [ ]:
# pickle 객체를 다시 item_texts로 복구

import pickle

# 파일을 읽기 모드로 열어서 pickle 객체를 로드합니다.
with open('item_texts.pkl', 'rb') as file:
    item_texts = pickle.load(file)

In [ ]:
# 2. SBERT로 임베딩 (두 텍스트 모두 동일 모델 사용)
sbert_model = SentenceTransformer("jhgan/ko-sroberta-multitask")

# 채용 공고 텍스트 임베딩 (결과 shape: (n_items, embedding_dim))
item_embeddings = sbert_model.encode(item_texts, normalize_embeddings=True)

# 3. LightFM이 요구하는 sparse matrix 형태로 변환
item_features = csr_matrix(item_embeddings)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
item_embeddings

array([[-0.05796488, -0.03381813, -0.00351672, ..., -0.00268984,
         0.03758674, -0.01541409],
       [-0.0583287 , -0.04616947, -0.04142432, ..., -0.04087724,
         0.00271495, -0.00012529],
       [-0.03172692,  0.0088002 , -0.00483126, ...,  0.00544613,
        -0.01983449, -0.04361816],
       ...,
       [-0.05137302, -0.00636557, -0.04813565, ...,  0.00869969,
         0.03064876, -0.02054955],
       [-0.01567443, -0.03628654,  0.00987602, ...,  0.03854228,
         0.00948803, -0.03832911],
       [-0.06125489, -0.00280513,  0.00106482, ...,  0.01524567,
        -0.02472108, -0.02471505]], dtype=float32)

In [ ]:
item_features

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 267264 stored elements and shape (348, 768)>

In [ ]:
# 1. user_text, 3. interaction_data : 구글폼에서 입력받아서 구글 스프레드시트 가져오기

# Google Sheets API를 사용할 범위 지정
scopes = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

# 기본 인증을 사용하는 경우
gc = gspread.authorize(google.auth.default(scopes=scopes)[0])

# 스프레드시트 열기 (문서 URL이나 제목을 사용)
spreadsheet = gc.open("구직자 응답")  # 스프레드시트 제목 사용
worksheet = spreadsheet.sheet1  # 첫 번째 시트 선택

# 데이터 가져오기
data = worksheet.get_all_records()
job_seekers = pd.DataFrame(data)

# 구직자 프로필 텍스트 생성
def create_profile_text(row):
    profile = f"직무: {job_seekers.loc[row, '직무']} 경력: {job_seekers.loc[row, '경력']} 자격증: {job_seekers.loc[row, '자격증']} 전공: {job_seekers.loc[row, '전공']} 기술: {job_seekers.loc[row, '기술']}"
    return profile

n = 2 # 구글 시트에서 몇번째 사람 데이터인지
user_text = create_profile_text(n)

input_str = job_seekers.loc[n, '관심 있는 공고 인덱스']

# 정규표현식을 사용하여 문자열에서 숫자(연속된 숫자들)를 추출합니다.
numbers = re.findall(r'\d+', input_str)

# 추출된 문자열 숫자들을 정수로 변환하고, ('user_1', 숫자) 형태의 튜플로 리스트를 생성합니다.
interaction_data = [('user_1', int(num)) for num in numbers]

user_text

SpreadsheetNotFound: <Response [200]>

In [ ]:
# 2. SBERT로 임베딩 (두 텍스트 모두 동일 모델 사용)
sbert_model = SentenceTransformer("jhgan/ko-sroberta-multitask")

# 유저 텍스트 임베딩 (결과 shape: (1, embedding_dim))
user_embedding = sbert_model.encode([user_text], normalize_embeddings=True)

# 3. LightFM이 요구하는 sparse matrix 형태로 변환
user_features = csr_matrix(user_embedding)
item_features = csr_matrix(item_embeddings)

# 4. Dataset 생성
# 유저는 한 명, 아이템은 채용 공고 리스트의 인덱스로 사용
dataset = Dataset()
dataset.fit(users=['user_1'], items=list(range(len(item_texts))))

# 5. interaction 생성
(interactions, _) = dataset.build_interactions(interaction_data)

# 6. LightFM 모델 학습 (user_features와 item_features 모두 사용)
model = LightFM(loss='warp')
model.fit(interactions, user_features=user_features, item_features=item_features,
          epochs=30, num_threads=2)

# 7. 유저에 대해 모든 공고 점수 예측 후 추천 순위 도출
scores = model.predict(0, np.arange(len(item_texts)), user_features=user_features, item_features=item_features)
recommended_order = np.argsort(-scores)

print("<추천 결과>")
for idx in recommended_order[:5]:
    print(f"{idx}: {recruit_df.loc[idx, '제목']} (score: {scores[idx]:.4f})")